In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124\run_objective_stackedtabnet_output_dim_8


'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [2]:
# tensorflow2.0 + kerasでGPUメモリの使用量を抑える方法(最小限だけ使うように設定)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
import sys
sys.path.append(r'C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124')
import moa_MLPs_funcs

Tensorflow version 2.3.1
n_classes: 206


In [5]:
import optuna


def objective(trial):
    model_type = "stacked_tabnet"
    
    params = dict(
        epsilon=1e-05,
        feature_columns=None,  # データセットのTensorflow特徴列
        virtual_batch_size=None,  # 仮想バッチサイズ。全体のバッチサイズは virtual_batch_size の整数倍じゃないとだめらしい
        num_layers=2,  # 重ねるTabNetsの数
        num_decision_steps=1,  # decision stepsの数
        norm_type="batch",  # 正規化のタイプ
        num_groups=-1,  # group normarizaionのグループの数。よくわからん
        batch_momentum=0.9, # 仮想バッチのMomentum。よくわからん
        relaxation_factor=trial.suggest_categorical("gamma", [1.0, 1.2, 1.5, 2.0]),  # 論文の探索範囲
        sparsity_coefficient=trial.suggest_categorical("lambda_sparse", [0, 0.000001, 0.0001, 0.001, 0.01, 0.1]),  # 論文の探索範囲。sparsity正則化
    )
    # feature_dim must be larger than output dim
    # feature_dim must be a list of length `num_layers`
    params["feature_dim"] = trial.suggest_categorical("Na", [128, 256, 512, 768, 1024, 1536, 2048])  # feature transformation block
    
    params["output_dim"] = 8  # params["feature_dim"]  # int(trial.suggest_uniform("Nd", params["feature_dim"] // 8, params["feature_dim"]))  # decision step

    print("-" * 100)
    print(f"params: {params}")

    oof_score, Y_pred = moa_MLPs_funcs.train_and_evaluate(
        model_type=model_type, params=params
    )

    return oof_score

In [6]:
%%time

import warnings
warnings.filterwarnings('ignore')

#n_trials = 100
n_trials = 30
#n_trials = 3

study = optuna.create_study(
    study_name="study",
    storage=f"sqlite:///study.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1),
)
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe().to_csv(f"objective_history.csv", index=False)
with open(f"objective_best_params.txt", mode="w") as f:
    f.write(str(study.best_params))
print(f"\nstudy.best_params:\n{study.best_params}")
print(f"\nstudy.best_value:\n{study.best_value}")

[I 2020-11-26 17:31:27,733] A new study created in RDB with name: study


----------------------------------------------------------------------------------------------------
params: {'epsilon': 1e-05, 'feature_columns': None, 'virtual_batch_size': None, 'num_layers': 2, 'num_decision_steps': 1, 'norm_type': 'batch', 'num_groups': -1, 'batch_momentum': 0.9, 'relaxation_factor': 1.2, 'sparsity_coefficient': 0.001, 'feature_dim': 1024, 'output_dim': 8}

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only

Epoch 41/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0219
Epoch 42/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0218
Epoch 43/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0217
Epoch 44/50
19/19 - 1s - loss: 0.0226 - val_loss: 0.0217
Epoch 45/50
19/19 - 1s - loss: 0.0225 - val_loss: 0.0216
Epoch 46/50
19/19 - 1s - loss: 0.0225 - val_loss: 0.0215
Epoch 47/50
19/19 - 1s - loss: 0.0225 - val_loss: 0.0214
Epoch 48/50
19/19 - 1s - loss: 0.0224 - val_loss: 0.0214
Epoch 49/50
19/19 - 1s - loss: 0.0224 - val_loss: 0.0213
Epoch 50/50
19/19 - 1s - loss: 0.0223 - val_loss: 0.0212
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
h

Epoch 42/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0227
Epoch 43/50
19/19 - 1s - loss: 0.0229 - val_loss: 0.0227
Epoch 44/50
19/19 - 1s - loss: 0.0230 - val_loss: 0.0227
Epoch 45/50
19/19 - 1s - loss: 0.0230 - val_loss: 0.0226
Epoch 46/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0226
Epoch 47/50
19/19 - 1s - loss: 0.0229 - val_loss: 0.0226
Epoch 48/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0225
Epoch 49/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0225
Epoch 50/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0225
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epo

[I 2020-11-26 17:36:59,933] Trial 0 finished with value: 0.018900315669853476 and parameters: {'gamma': 1.2, 'lambda_sparse': 0.001, 'Na': 1024}. Best is trial 0 with value: 0.018900315669853476.


...

oof_auc_score: 0.5878831642010066
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
params: {'epsilon': 1e-05, 'feature_columns': None, 'virtual_batch_size': None, 'num_layers': 2, 'num_decision_steps': 1, 'norm_type': 'batch', 'num_groups': -1, 'batch_momentum': 0.9, 'relaxation_factor': 1.2, 'sparsity_coefficient': 0, 'feature_dim': 128, 'output_dim': 8}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch

Epoch 43/50
19/19 - 1s - loss: 0.0229 - val_loss: 0.0223
Epoch 44/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0222
Epoch 45/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0221
Epoch 46/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0221
Epoch 47/50
19/19 - 1s - loss: 0.0229 - val_loss: 0.0221
Epoch 48/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0220
Epoch 49/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0220
Epoch 50/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0220
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/50

If you intended to run this layer in float32, yo

Epoch 44/50
19/19 - 1s - loss: 0.0230 - val_loss: 0.0226
Epoch 45/50
19/19 - 1s - loss: 0.0230 - val_loss: 0.0226
Epoch 46/50
19/19 - 1s - loss: 0.0228 - val_loss: 0.0225
Epoch 47/50
19/19 - 1s - loss: 0.0229 - val_loss: 0.0225
Epoch 48/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0225
Epoch 49/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0224
Epoch 50/50
19/19 - 1s - loss: 0.0227 - val_loss: 0.0224
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warni

[I 2020-11-26 17:48:53,210] Trial 2 finished with value: 0.01904909150771326 and parameters: {'gamma': 1.2, 'lambda_sparse': 0, 'Na': 128}. Best is trial 1 with value: 0.018723874071894894.


oof_auc_score: 0.5709790679418182
----------------------------------------------------------------------------------------------------

study.best_params:
{'Na': 768, 'gamma': 1.0, 'lambda_sparse': 1e-06}

study.best_value:
0.018723874071894894
Wall time: 17min 25s
